# Open Router

### Here I will test the models that are mainly related to image generation

## Example 1: (Text to Text using OpenRouter)

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
load_dotenv(override=True)
or_api_key = os.getenv("OP_API")

if not or_api_key:
      print("Not Found")
else:
      print("API Key Found")

In [3]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=f"{or_api_key}", # Get this from openrouter.ai
    default_headers={
        "HTTP-Referer": "http://localhost:3000", # Optional: Your site URL for rankings
        "X-Title": "My Technical Assistant",   # Optional: Your app name
    }
)

In [4]:
def message_openrouter(prompt, model_id="deepseek/deepseek-r1-0528:free"):
    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [5]:
# print(message_openrouter("Explain quantum computing in one sentence."))

## Example 2 : (Text to Image generation using OpenRouter)

In [5]:
import base64
from openai import OpenAI

In [6]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=f"{or_api_key}",
)

In [7]:
g=0

In [ ]:
def generate_nano_banana_image(prompt):
    try:
        # Use the latest stable ID
        model_id = "google/gemini-2.5-flash-image" 
        
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=2000,
            extra_body={
                "modalities": ["image", "text"],
                "image_config": {"aspect_ratio": "16:9"}
            }
        )

        # Access the message object
        message = response.choices[0].message
        
        # OpenRouter-specific field check
        if hasattr(message, 'images') and message.images:
            for i, img_obj in enumerate(message.images):
                # THE FIX: Drill down into the specific nested structure
                # We check 'url' and 'image_url' -> 'url'
                raw_url = img_obj.get('url') or img_obj.get('image_url', {}).get('url')
                
                if not raw_url:
                    print(f"Skipping index {i}: No URL found in structure: {img_obj}")
                    continue

                if "base64," in raw_url:
                    # Extract the pure base64 data
                    base64_data = raw_url.split("base64,")[1]
                    filename = f"output_image_{g}.png"
                    g=g+1
                    
                    with open(filename, "wb") as f:
                        f.write(base64.b64decode(base64_data))
                    print(f"Successfully saved: {filename}")
                else:
                    print(f"Warning: Model returned a link instead of raw data: {raw_url}")
            
            return "Task completed."
        else:
            return f"No images found. Response text: {message.content}"

    except Exception as e:
        return f"CRITICAL ERROR: {str(e)}"



In [ ]:
# Execute
print(generate_nano_banana_image("Lounge chair, thick glass panels, aluminum tube frame, brown leather sling seat, photorealistic studio lighting, dark background, 8k."))